In [1]:
import tensorflow as tf
from tensorflow.keras import utils, models, layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from gui import create_model_gui

In [5]:
model, params_dict = create_model_gui()
params_dict

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 1)                 27        
                                                                 
 dense_12 (Dense)            (None, 2)                 4         
                                                                 
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


{'Season': '2000', 'loss function': 'Cross Entropy'}

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 1)                 27        
                                                                 
 dense_12 (Dense)            (None, 2)                 4         
                                                                 
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________
